In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextStreamer
from trl import SFTTrainer
from transformers import TrainingArguments

    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.2.1+cu121)
    Python  3.10.14 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [ ]:
import json
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
from google.colab import files

In [ ]:
# device = 'cuda'

In [ ]:
CNN_tokenizer = AutoTokenizer.from_pretrained("Sardean-UChicago/CNN_Mistral-7B-Headline-QLoRA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
CNN_model = AutoModelForCausalLM.from_pretrained("Sardean-UChicago/CNN_Mistral-7B-Headline-QLoRA")

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
test_CNN_instructions_df = pd.read_json('test_CNN_instructions.jsonl', lines=True)
test_CNN_bodies_df = pd.read_json('test_CNN_bodies.jsonl', lines=True)
test_CNN_titles_df = pd.read_json('test_CNN_titles.jsonl', lines=True)

test_CNN_bodies_df.head()

,text
0,Criticism of Justice Clarence Thomas’ refusal ...
1,Tuesday marked the worst performance this prim...
2,The House select committee investigating Janua...
3,"CNN, joined by The Washington Post, NBC News a..."
4,Despite a renewed commitment on civilian casua...


In [ ]:
true_CNN_titles = test_CNN_titles_df['text'] # true titles accompanying text
true_CNN_titles_100 = true_CNN_titles[:100]
true_CNN_titles_100

0     What to know about the Justice Clarence Thomas...
1     The GOP is still closer to Trump than it is to...
2     Jan. 6 hearing: Here’s what to watch for from ...
3     CNN and other news outlets ask court to unseal...
4     Pentagon doesn’t hold anyone accountable for 2...
                            ...                        
95    McCarthy to lead GOP lawmakers, including Marj...
96    DOJ asks judge to force Trump White House lawy...
97    Exclusive: Musk’s SpaceX says it can no longer...
98    Herschel Walker’s defeat delivers another blow...
99          The 2022 election is what we thought it was
Name: text, Length: 100, dtype: object

In [ ]:
# true_CNN_titles_10samples = true_CNN_titles[:10]
# #true_titles_500 = true_titles[:500]
# true_CNN_titles_10samples

In [ ]:
def LLM_generate_test(model,tokenizer,instructions_df,bodies_df,titles_df):
  generated_texts = []
  for i in range(len(instructions_df)):
    prompt = f"""Give a title to this writing in my style

### Text
{bodies_df['text'].iloc[i]}

### Title

"""

    inputs = tokenizer(prompt,
        return_tensors="pt",
        truncation=True,
        max_length=163804
    )
    outputs = model.generate(
        **inputs,
        max_length=30000,
        num_return_sequences=1,
        use_cache=True
    )
    generated_text = tokenizer.batch_decode(outputs, skin_special_tokens=True)
    generated_texts.append(generated_text)
    if i%10 ==0:
      print(f"Epoch {i}")
  return generated_texts

In [ ]:
# sample_CNN_instructions_df = test_CNN_instructions_df[:10]
# sample_CNN_bodies_df = test_CNN_bodies_df[:10]
# sample_CNN_titles_df = test_CNN_titles_df[:10]

In [ ]:
# sample_generate = LLM_generate_test(CNN_model,CNN_tokenizer,sample_CNN_instructions_df,sample_CNN_bodies_df,sample_CNN_titles_df)

In [ ]:
def title_parse(generated_text):
  titles = []
  for item in generated_text:
    for text in item:
      if '### Title' in text:
        title = text.split('### Title')[1].strip().split('</s>')[0].strip()
        titles.append(title)
  return titles

In [ ]:
# pred_titles = title_parse(sample_generate)

In [ ]:
def create_df(true_titles,pred_titles):
  new_df = pd.DataFrame(
      {
          'true_titles': true_titles,
          'pred_titles': pred_titles
      }
  )

  return new_df

In [ ]:
# pred_titles

In [ ]:
# create_df(true_CNN_titles_10samples,pred_titles)

In [ ]:
def test_pipeline(model,tokenizer,instructions_df,bodies_df,titles_df):
  true_titles = titles_df['text']
  generated_text = LLM_generate_test(model,tokenizer,instructions_df,bodies_df,titles_df)
  pred_titles = title_parse(generated_text)
  new_df = create_df(true_titles,pred_titles)
  return new_df

In [ ]:
CNN_model_on_CNN_test_100 = test_pipeline(CNN_model,CNN_tokenizer,
                                     test_CNN_instructions_df[:100],
                                     test_CNN_bodies_df[:100],
                                     test_CNN_titles_df[:100])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 10


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 80


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 90


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
CNN_model_on_CNN_test_100.to_csv('CNN_model_on_CNN_test_100.csv',index=False)

In [ ]:
files.download('CNN_model_on_CNN_test_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt = """Give a title to this writing in my style

# ### Text

# Malaysia and Indonesia have issued separate statements calling for restraint after Iran's retaliatory strikes on Israel.

# Both Southeast Asian countries, where Islam is the predominant religion, reiterated support for Palestinian rights and causes. Malaysia and Indonesia do not recognize Israel.

# "Malaysia strongly urges all parties in the Middle East region to refrain and exercise great caution and not to escalate the already tense situation," its Ministry of Foreign Affairs said Sunday.

# ### Title

# """

In [ ]:
# test_prompt1 = f"""Give a title to this writing in my style

# ### Text
# {CNN_prompt1[:1000]}

# ### Title

# """
# test_prompt1

In [ ]:
# inputs = CNN_tokenizer(test_prompt1,
#     return_tensors="pt",
#     truncation=True,
#     max_length=163804
# )


# outputs = CNN_model.generate(
#     **inputs,
#     max_length=30000,
#     num_return_sequences=1,
#     use_cache=True
# )

# #generated_text = CNN_tokenizer.batch_decode(outputs, skin_special_tokens=True)
# CNN_generated_text1 = CNN_tokenizer.batch_decode(outputs, skin_special_tokens=True)

In [ ]:
# len(prompt), len(CNN_prompt1)

In [ ]:
FOX_tokenizer = AutoTokenizer.from_pretrained("Sardean-UChicago/FOX_Mistral-7B-Headline-QLoRA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
FOX_model = AutoModelForCausalLM.from_pretrained("Sardean-UChicago/FOX_Mistral-7B-Headline-QLoRA")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# sample_generate_FOX = LLM_generate_test(FOX_model,FOX_tokenizer,sample_CNN_instructions_df,sample_CNN_bodies_df,sample_CNN_titles_df)

In [ ]:
# pred_titles_sample_FOX = title_parse(sample_generate_FOX)

In [ ]:
# create_df(true_CNN_titles_10samples,pred_titles_sample_FOX)

In [ ]:
test_FOX_instructions_df = pd.read_json('test_FOX_instructions.jsonl', lines=True)
test_FOX_bodies_df = pd.read_json('test_FOX_bodies.jsonl', lines=True)
test_FOX_titles_df = pd.read_json('test_FOX_titles.jsonl', lines=True)

test_FOX_bodies_df.head()

,text
0,FIRST ON FOX: Utah Senate candidate Evan McMul...
1,"Blake Masters, the Arizona Republican nominee ..."
2,That's the message U. S. Senate candidate Adam...
3,It shouldn't be this close in Ohio. With one w...
4,American democracy and the American people are...


In [ ]:
FOX_true_titles = test_FOX_titles_df['text'] # true titles accompanying text

In [ ]:
FOX_model_on_FOX_test_100 = test_pipeline(FOX_model,FOX_tokenizer,
                                     test_FOX_instructions_df[:100],
                                     test_FOX_bodies_df[:100],
                                     test_FOX_titles_df[:100])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 10


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 80


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 90


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
FOX_model_on_FOX_test_100.to_csv('FOX_model_on_FOX_test_100.csv',index=False)

In [ ]:
files.download('FOX_model_on_FOX_test_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
CNN_model_on_FOX_test_100 = test_pipeline(CNN_model,CNN_tokenizer,
                                     test_FOX_instructions_df[:100],
                                     test_FOX_bodies_df[:100],
                                     test_FOX_titles_df[:100])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 10


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 80


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 90


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
CNN_model_on_FOX_test_100.to_csv('CNN_model_on_FOX_test_100.csv',index=False)

In [ ]:
files.download('CNN_model_on_FOX_test_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
FOX_model_on_CNN_test_100 = test_pipeline(FOX_model,FOX_tokenizer,
                                     test_CNN_instructions_df[:100],
                                     test_CNN_bodies_df[:100],
                                     test_CNN_titles_df[:100])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 10


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 80


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Epoch 90


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
FOX_model_on_CNN_test_100.to_csv('FOX_model_on_CNN_test_100.csv',index=False)

In [ ]:
files.download('FOX_model_on_CNN_test_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>